In [2]:
# import libraries
import pandas as pd
import numpy as np

# from uszipcode import Zipcode, SearchEngine

import warnings
warnings.filterwarnings("ignore")

### Import major incidents files

These are separated into Northern and Southern California Area Coordination Centers. Each report had to be downloaded separately by year. I will pull them all into individual variables which can then be concatenated together to create one inclusive incident report dataframe.

This data includes all major wildfire incidents, defined as larger than 100 acres.

In [ ]:
fire_df_nc_19 = pd.read_excel('../claire_data/Significant Incident Summary Spreadsheet by GACC norcal 2019.xlsx')
fire_df_nc_18 = pd.read_excel('../claire_data/Significant Incident Summary Spreadsheet by GACC norcal 2018.xlsx')
fire_df_nc_17 = pd.read_excel('../claire_data/Significant Incident Summary Spreadsheet by GACC norcal 2017.xlsx')
fire_df_nc_16 = pd.read_excel('../claire_data/Significant Incident Summary Spreadsheet by GACC norcal 2016.xlsx')
fire_df_nc_15 = pd.read_excel('../claire_data/Significant Incident Summary Spreadsheet by GACC norcal 2015.xlsx')
fire_df_nc_14 = pd.read_excel('../claire_data/Significant Incident Summary Spreadsheet by GACC norcal 2014.xlsx')

fire_df_sc_19 = pd.read_excel('./claire_data/Significant Incident Summary Spreadsheet by GACC socal 2019.xlsx')
fire_df_sc_18 = pd.read_excel('./claire_data/Significant Incident Summary Spreadsheet by GACC socal 2018.xlsx')
fire_df_sc_17 = pd.read_excel('./claire_data/Significant Incident Summary Spreadsheet by GACC socal 2017.xlsx')
fire_df_sc_16 = pd.read_excel('./claire_data/Significant Incident Summary Spreadsheet by GACC socal 2016.xlsx')
fire_df_sc_15 = pd.read_excel('./claire_data/Significant Incident Summary Spreadsheet by GACC socal 2015.xlsx')
fire_df_sc_14 = pd.read_excel('./claire_data/Significant Incident Summary Spreadsheet by GACC socal 2014.xlsx')

In [ ]:
fires_df = pd.concat([fire_df_nc_14, fire_df_sc_14, 
                      fire_df_nc_15, fire_df_sc_15,
                      fire_df_nc_16, fire_df_sc_16,
                      fire_df_nc_17, fire_df_sc_17,
                      fire_df_nc_18, fire_df_sc_18, 
                      fire_df_nc_19, fire_df_sc_19]).reset_index(drop=True)

In [ ]:
fires_df.head(2)

In [ ]:
fires_df.info()

### Data Cleaning

There are several features to be updated here. A few of them are:
- Making all incident names uppercase to match the shapefile data
- Year: extract year from start date
- Costs: remove all fires that don't have an associated cost
- Incident Type: remove all incidents that are not classified as wildfires
- State: remove all fires outside of CA (a few fires in HI are under Southern CA's jurisdiction)
- County: use 'uszipcode' library to extract county using latitude and longitude measures

In [ ]:
fires_df['Fire Name'] = fires_df['Incident Name'].str.upper()

In [ ]:
fires_df['start_date'] = pd.to_datetime(fires_df['Start Date'], utc=True).dt.date

In [ ]:
fires_df['year'] = pd.to_datetime(fires_df['Start Date'], utc=True).dt.year

In [ ]:
fires_df.head(2)

In [ ]:
fires_df.dropna(subset=['Costs'], axis=0, inplace=True)

In [ ]:
fires_df['Incident Type'].value_counts()

In [ ]:
# Drop all non-wildfire incidents
fires_df = fires_df[fires_df['Incident Type'] == 'WF']

In [ ]:
fires_df['State-Unit'].unique()

In [ ]:
fires_df = fires_df[fires_df['State-Unit'].str.startswith('CA')]

In [ ]:
fires_df.reset_index(drop=True, inplace=True)

In [ ]:
# Code adapted from Emiko Sano

def county_name(lat, long, radius=100):
    '''Takes in latitude and longitude (as an iterable) and returns the closest county name for the given coordinates'''
    lat, long = lat, long * -1
    search = SearchEngine()
    result = search.by_coordinates(lat, long)
    # get the county name
    try:
        county = result[0].county
        # this is in 'XXXX County', so fix the format so that it's only the county name without 'County' at the end
        county = county.split()
        # get everything but the last item (which is 'County')
        county = county[:-1]
        # return a string
        return ' '.join(county)
    except:
        print(f'Something went wrong. Check your coordinates: {lat, long}') 

In [ ]:
def county_column(df):
    '''Applies the county_name function to latitude and longitude columns in order to create a new county column'''
    county_list = []
    for num in range(len(df)):
        latitude, longitude = float(fires_df.iloc[num, 7]), float(fires_df.iloc[num, 8])
        county_list.append(county_name(latitude, longitude))
        
    return county_list

In [ ]:
fires_df['county'] = county_column(fires_df)

In [ ]:
fires_df[fires_df['county'].isna()]

In [ ]:
# Manually correcting counties that had NaN values
fires_df.loc[45, 'county'] = 'San Diego'
fires_df.loc[61, 'county'] = 'Los Angeles'
fires_df.loc[136, 'county'] = 'San Diego'
fires_df.loc[328, 'county'] = 'San Diego'

In [ ]:
fires_df_clean = fires_df.drop(columns = ['State-Unit', 'Start Date', 
                               'IC Name', 'Team Type', 'Anticipated Containment / Completion Date'])
fires_df_clean.head(2)

In [ ]:
fires_df_clean.info()

In [ ]:
fires_df_clean.describe()

In [ ]:
fires_df_clean = fires_df_clean[(fires_df_clean['Size'] > 100) & (fires_df_clean['Costs'] > 0)]

In [ ]:
fires_df_clean.to_csv('./claire_data/fires_compiled.csv', index=False)

### Acres Burned Over Time

This will be used for time series forecasting.

In [4]:
acres_df = pd.read_csv('../claire_data/California_Fire_Perimeters.csv')

In [5]:
acres_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12202 entries, 0 to 12201
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OBJECTID      12202 non-null  int64  
 1   YEAR_         12140 non-null  float64
 2   STATE         12148 non-null  object 
 3   AGENCY        12202 non-null  object 
 4   UNIT_ID       12199 non-null  object 
 5   FIRE_NAME     12174 non-null  object 
 6   INC_NUM       11720 non-null  object 
 7   ALARM_DATE    8214 non-null   object 
 8   CONT_DATE     3375 non-null   object 
 9   CAUSE         12190 non-null  float64
 10  COMMENTS      1886 non-null   object 
 11  REPORT_AC     3836 non-null   float64
 12  GIS_ACRES     12202 non-null  float64
 13  C_METHOD      3759 non-null   float64
 14  OBJECTIVE     12146 non-null  float64
 15  FIRE_NUM      10952 non-null  object 
 16  GlobalID      12202 non-null  object 
 17  SHAPE_Length  12202 non-null  float64
 18  SHAPE_Area    12202 non-nu

In [6]:
acres_df.dropna(subset=['YEAR_'], inplace=True)

In [7]:
acres_df = acres_df[(acres_df['YEAR_'] > 1970) & (acres_df['GIS_ACRES'] > 100)]

In [8]:
acres_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6086 entries, 0 to 12201
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   OBJECTID      6086 non-null   int64  
 1   YEAR_         6086 non-null   float64
 2   STATE         6032 non-null   object 
 3   AGENCY        6086 non-null   object 
 4   UNIT_ID       6083 non-null   object 
 5   FIRE_NAME     6068 non-null   object 
 6   INC_NUM       5924 non-null   object 
 7   ALARM_DATE    5526 non-null   object 
 8   CONT_DATE     3252 non-null   object 
 9   CAUSE         6074 non-null   float64
 10  COMMENTS      1079 non-null   object 
 11  REPORT_AC     3207 non-null   float64
 12  GIS_ACRES     6086 non-null   float64
 13  C_METHOD      3351 non-null   float64
 14  OBJECTIVE     6057 non-null   float64
 15  FIRE_NUM      4930 non-null   object 
 16  GlobalID      6086 non-null   object 
 17  SHAPE_Length  6086 non-null   float64
 18  SHAPE_Area    6086 non-null